In [24]:
import re
import scipy
import numpy as np
from scipy import stats
from collections import OrderedDict, defaultdict


    
def norm(sam, stan):
    # get the normalization rdefaatio
    geex, cose, coan, coto, struc, pseudo, mir = 0, 0, 0, 0, 0, 0, 0
    file = sam+'_all/sum_'+sam+'_all.txt'
    with open (file) as f:
        for line in f:
            #line = re.sub(r'\s+$', r'', line)
            line = line.rstrip()
            if re.search(r'^geex\t\S+\t\S+\t\S+', line):
                mch = re.search(r'^geex\t\S+\t\S+\t(\S+)', line) # all
                geex = float(mch.group(1))
            if (re.search(r'^protein_coding\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'^protein_coding\t(\S+)\t(\S+)\t(\S+)', line)
                cose = float(mch.group(1)); coan = float(mch.group(2)); coto = float(mch.group(3))
            if (re.search(r'^struc\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'^struc\t(\S+)\t\S+\t\S+', line) # sense
                struc = float(mch.group(1))            
            if (re.search(r'^pseudogene\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'pseudogene\t(\S+)\t\S+\t\S+', line) # sense
                pseudo = float(mch.group(1))            
            if (re.search(r'^miRNA\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'miRNA\t(\S+)\t\S+\t\S+', line) # sense
                mir = float(mch.group(1))            
            if (re.search(r'^premiRNA\t\S+\t\S+\t\S+', line)):
                mch = re.search(r'premiRNA\t(\S+)\t\S+\t\S+', line) # sense
                premir = float(mch.group(1))
            if (re.search(r'^exex', line)):
                break

    print ('''total %.2f
              sense coding %.2f
              anti coding %.2f
              total coding %.2f
              sense struc %.2f
              sense pseudo %.2f
              sense miRNA %.2f\n''' % (geex, cose, coan, coto, struc, pseudo, mir))
        
    if (stan == 'mir'):
        ratio = 1000000/mir
    elif (stan == 'cose'):
        ratio = 5000000/cose
    elif (stan == 'coan'):
        ratio = 2000000/coan
    elif (stan == 'coto'):
        ratio = 5000000/coto
    elif (stan == 'tot'):
        ratio = 5000000/geex
    
    print ('Normalization ratio', ratio, '\n\n')
    
    return ratio

ano = {}

with open('Aedes-aegypti-Liverpool_TRANSCRIPTS_AaegL3.3.fa', 'rt') as f:
    for line in f:
        if line.startswith('>'):
            line = line.rstrip()
            line = re.sub('>', '', line)
            lst = line.split()
            gene = lst[0].split('-')
            geneid = gene[0]+'\t'+lst[0]
            lst.pop(0)
            info = ' '.join(lst)
            ano[geneid] = info

coding = OrderedDict(defaultdict(list))

stan = 'cose'

ctrl = '032816ctrl'
treat = '032816LC506hr'


wt = input('WT samples seperated with comma (3 replicates):')
wt = wt.rstrip()
mut = input('Mutant samples seperated with comma (3 replicates):')
mut = mut.rstrip()

samples = re.split(r'\s*\,+\s*', wt) + re.split(r'\s*\,+\s*', mut)

for sam in samples:
    ra = norm(sam, stan)
    in_file = sam+'_all/sum_'+sam+'_all_17_cdna_protein_coding.txt'
    with open(in_file, 'rt') as f:
        for line in f:
            line = line.rstrip()
            lst = line.split()
            gene = lst[0]+'\t'+lst[1]
            if gene not in coding.keys():
                coding[gene] = []
            coding[gene].append(float(lst[2])*ra)

fh1 = open('T_test_'+ctrl+'_'+treat+'_'+'005.txt', 'wt')
fh2 = open('T_test_'+ctrl+'_'+treat+'.txt', 'wt')
fh1.write('Gene\tTranscript\t')
fh2.write('Gene\tTranscript\t')

for sam in samples:
    fh1.write(sam+'\t')
    fh2.write(sam+'\t')

fh1.write('WT mean\tMUT mean\tp-value\n')
fh2.write('WT mean\tMUT mean\tp-value\n')

for k,v in coding.items():
    x = v[0:3]
    y = v[3:6]
    x_m = round(np.mean(x), 3)
    y_m = round(np.mean(y), 3)

    v1 = str(v[0])
    v2 = str(v[1])
    v3 = str(v[2])
    v4 = str(v[3])
    v5 = str(v[4])
    v6 = str(v[5])

    
    test = stats.ttest_ind(x,y, equal_var=True)
    t = round(test[0], 3)
    p = round(test[1], 3)
    if p < 0.05:
        fh1.write(k+'\t'+v1+'\t'+v2+'\t'+v3+'\t'+v4+'\t'+v5+'\t'+v6+'\t'+str(x_m)+'\t'+str(y_m)+'\t'+str(p)+'\t'+ano[k]+'\n')
    
    fh2.write(k+'\t'+v1+'\t'+v2+'\t'+v3+'\t'+v4+'\t'+v5+'\t'+v6+'\t'+str(x_m)+'\t'+str(y_m)+'\t'+str(p)+'\t'+ano[k]+'\n')
    
fh1.close()
fh2.close()



            
            


WT samples seperated with comma (3 replicates):032816ctrl1,032816ctrl2,032816ctrl4
Mutant samples seperated with comma (3 replicates):032816LC506hr1,032816LC506hr2,032816LC506hr4
total 4862074.00
sense coding 2554289.00
anti coding 75315.00
total coding 2629604.00
sense struc 803684.00
sense pseudo 43.00
sense miRNA 93.00

Normalization ratio 1.9574918891323574 


total 3198519.00
sense coding 1715226.00
anti coding 48871.00
total coding 1764097.00
sense struc 535385.00
sense pseudo 23.00
sense miRNA 44.00

Normalization ratio 2.915067752004692 


total 2703967.00
sense coding 1418332.00
anti coding 40953.00
total coding 1459285.00
sense struc 398667.00
sense pseudo 19.00
sense miRNA 47.00

Normalization ratio 3.5252677088298086 


total 3000762.00
sense coding 1648936.00
anti coding 50196.00
total coding 1699132.00
sense struc 449717.00
sense pseudo 49.00
sense miRNA 28.00

Normalization ratio 3.0322583775234455 


total 4136356.00
sense coding 2308214.00
anti coding 71519.00
total co

In [ ]:
032816ctrl1,032816ctrl2,032816ctrl4
032816LC506hr1,032816LC506hr2,032816LC506hr4

In [20]:
lst = ['a', 'b', 'c']
lst.pop(0)
info = ' '.join(lst)
print (info)

b c


In [12]:
from scipy import stats
rvs1 = stats.norm.rvs(loc=5,scale=10,size=50)
rvs1

x = [1,2,3,4,5,6]
y = [9,10,11]

test = stats.ttest_ind(x,y)
p = test[1]
print (p)

0.000899300636929


In [16]:
import numpy as np
np.mean(x)

3.5

In [5]:
import os
os.getcwd()

'/Users/DONG/Documents/Career/Python2/Python_lab/jianwu'

In [10]:
import re
ano = {}

i = 0

with open('mRNAseq/Aedes-aegypti-Liverpool_TRANSCRIPTS_AaegL3.3.fa', 'rt') as f:
    for line in f:
        if line.startswith('>'):
            i += 1
            if i > 10:
                break
                
            line = line.rstrip()
            line = re.sub('>', '', line)
            lst = line.split()
            #gene = lst[0].split('-')
            #geneid = gene[0]+'\t'+lst[0]
            #ano[geneid] = lst[1]
            print (line)

AAEL000001-RA  bystin|protein_coding|supercont1.1:4290724-4298425:-1|gene:AAEL000001
AAEL000002-RA  hypothetical protein|protein_coding|supercont1.1:1151653-1151850:1|gene:AAEL000002
AAEL000003-RA  conserved hypothetical protein|protein_coding|supercont1.1:3644561-3668187:1|gene:AAEL000003
AAEL000004-RA  glycosyltransferase|protein_coding|supercont1.1:2714033-2718806:-1|gene:AAEL000004
AAEL000005-RC  hypothetical protein|protein_coding|supercont1.1:4197253-4221452:1|gene:AAEL000005
AAEL000005-RB  hypothetical protein|protein_coding|supercont1.1:4208813-4221459:1|gene:AAEL000005
AAEL000006-RA  phosphoenolpyruvate carboxykinase|protein_coding|supercont1.1:4321657-4359569:-1|gene:AAEL000006
AAEL000006-RC  phosphoenolpyruvate carboxykinase|protein_coding|supercont1.1:4321517-4323668:-1|gene:AAEL000006
AAEL000006-RB  phosphoenolpyruvate carboxykinase|protein_coding|supercont1.1:4357192-4359569:-1|gene:AAEL000006
AAEL000007-RA  conserved hypothetical protein|protein_coding|supercont1.1:38491

In [6]:
import os
fls = os.listdir()
for fl in fls:
    if fl.endswith('.txt'):
        print (fl+'\n')

T_test_032816ctrl_032816LC5012hr.txt

T_test_032816ctrl_032816LC5012hr_005.txt

T_test_032816ctrl_032816LC5024hr.txt

T_test_032816ctrl_032816LC5024hr_005.txt

T_test_032816ctrl_032816LC506hr.txt

T_test_032816ctrl_032816LC506hr_005.txt

T_test_032816ctrl_032816R44F.txt

T_test_032816ctrl_032816R44F_005.txt

